In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator , img_to_array, load_img
from tensorflow.keras.optimizers import RMSprop
from keras.applications.vgg16 import preprocess_input

In [9]:
train = ImageDataGenerator(rescale = 1./255)
validation = ImageDataGenerator(rescale = 1./255)

In [24]:
train_dataset = train.flow_from_directory('Dataset/TrainSet',
                                         target_size = (128,128),
                                         batch_size = 32,
                                         class_mode = "categorical")
validation_dataset = validation.flow_from_directory('./Dataset/ValidationSet',
                                                   target_size = (128,128),
                                                   batch_size = 32,
                                                   class_mode = "categorical")

Found 240 images belonging to 12 classes.
Found 60 images belonging to 12 classes.


In [4]:
train_dataset.class_indices


{'bed': 0,
 'cow': 1,
 'father': 2,
 'full': 3,
 'have': 4,
 'police': 5,
 'sandwich': 6,
 'shirt': 7,
 'show': 8,
 'sick': 9,
 'water': 10,
 'wrong': 11}

In [5]:
train_dataset.classes

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11,
       11, 11, 11, 11, 11

# VGG Implementation

In [6]:
from keras.applications.vgg16 import VGG16
from glob import glob
from keras.models import Model
from tensorflow.keras.layers import Flatten, Dense

In [23]:
vgg = VGG16(input_shape=(128,128,3), weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False
  

  
  # useful for getting number of classes
folders = glob('/Dataset/ValidationSet/*')
  

# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0   

In [25]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit_generator(steps_per_epoch=100,generator=train_dataset, validation_data= validation_dataset, validation_steps=10,epochs=100,callbacks=[checkpoint,early])

Epoch 1/100


InvalidArgumentError:  Reduction axis -1 is empty in shape [32,0]
	 [[node ArgMax_1 (defined at \AppData\Local\Temp\ipykernel_25576\3066657587.py:4) ]] [Op:__inference_train_function_7265]

Function call stack:
train_function


# Second Model

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16,(3,3),activation = 'relu', input_shape=(512,512,3)),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    
                                    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu'),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    
                                    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu'),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    
                                    tf.keras.layers.Flatten(),
                                    
                                    tf.keras.layers.Dense(512,activation = 'relu'),
                                    
                                    tf.keras.layers.Dense(1,activation = 'sigmoid')
                                   ])

In [ ]:
model.compile(loss = 'binary_crossentropy',
             optimizer = RMSprop(learning_rate = 0.001),
             metrics= ['accuracy'])

In [ ]:
model_fit = model.fit(train_dataset,
                     steps_per_epoch = 5,
                     epochs = 10,
                     validation_data = validation_dataset)

In [ ]:
# dir_path = "Dataset/TestSet"
# for i in os.listdir(dir_path):
#     img = cv2.imread(dir_path + '//'+ i)
#     resized = cv2.resize(img,(512,512))
#     plt.imshow(img)
#     plt.show()
    
#     x = img_to_array(resized)
#     x = preprocess_input(x)
#     val = model.predict(x)
#     print(val)

In [ ]:
dir_path = "Dataset/TestSet"
for i in os.listdir(dir_path):
    my_image = load_img(dir_path + '//' + i , target_size=(512, 512))
    plt.imshow(my_image)
    plt.show()
    
    #preprocess the image
    my_image = img_to_array(my_image)
    my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
    my_image = preprocess_input(my_image)

    #make the prediction
    prediction = model.predict(my_image)
    print(prediction)